In [1]:
# # Auto reload
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from transformer_lens import HookedTransformer

from sparse_autoencoder.autoencoder.model import SparseAutoencoder
from sparse_autoencoder.dataset.dataloader import (
    collate_neel_c4_tokenized,
    create_dataloader,
)
from sparse_autoencoder.train.train import pipeline
import os

# Disable tokenizers forking
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
src_model = HookedTransformer.from_pretrained("NeelNanda/GELU_1L512W_C4_Code")
n_input_features = src_model.cfg.d_mlp

autoencoder = SparseAutoencoder(
    n_input_features=n_input_features,
    n_learned_features=n_input_features * 8,
    geometric_median_dataset=torch.zeros(n_input_features),
)
autoencoder

Loaded pretrained model NeelNanda/GELU_1L512W_C4_Code into HookedTransformer


SparseAutoencoder(
  (encoder): Sequential(
    (0): PreEncoderBias()
    (1): Linear(in_features=2048, out_features=16384, bias=True)
    (2): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=16384, out_features=2048, bias=True)
    (1): PostEncoderBias()
  )
)

In [4]:
dataloader = create_dataloader(
    "NeelNanda/c4-code-tokenized-2b",
    collate_fn=collate_neel_c4_tokenized,
    shuffle_buffer_size=1000,
    batch_size=2,
    num_workers=1,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

In [5]:
pipeline(
    src_model=src_model,
    autoencoder=autoencoder,
    activation_hook_point="blocks.0.mlp.hook_post",
    prompts_dataloader=dataloader,
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Moving model to device:  mps


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: alan-cooney. Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set th

logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048
logging 2048

/Users/alan/Documents/Repos/sparse_autoencoder/.venv/lib/python3.11/site-packages/torch/autograd/__init__.py:251: UserWarning: The operator 'aten::sgn.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


step 1
step 2
step 3
step 4
step 5
step 6
step 7
step 8
step 9
step 10
step 11
step 12
step 13
step 14
step 15
step 16
step 17
step 18
step 19
step 20
step 21
step 22
step 23
step 24
step 25
step 26
step 27
step 28
step 29
step 30
step 31
step 32
step 33
step 34
step 35
step 36
step 37
step 38
step 39
step 40
step 41
step 42
step 43
step 44
step 45
step 46
step 47
step 48
step 49
step 50
step 51
step 52
step 53
step 54
step 55
step 56
step 57
step 58
step 59
step 60
step 61
step 62
step 63
step 64
step 65
step 66
step 67
step 68
step 69
step 70
step 71
step 72
step 73
step 74
step 75
step 76
step 77
step 78
step 79
step 80
step 81
step 82
step 83
step 84
step 85
step 86
step 87
step 88
step 89
step 90
step 91
step 92
step 93
step 94
step 95
step 96
step 97
step 98
step 99
step 100
step 101
step 102
step 103
step 104
step 105
step 106
step 107
step 108
step 109
step 110
step 111
step 112
step 113
step 114
step 115
step 116
step 117
step 118
step 119
step 120
step 121
step 122
step 123
s

KeyboardInterrupt: 